## Buildings exposed statistics and maps

Notebook environment to rework the data from BRGM to parquet and gpkg


In [96]:
# Import standard packages
import os
import glob
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio
from itertools import product
#load_dotenv()

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

# use local or remote data dir
use_local_data = False
ds_dirname = "WP6"

if use_local_data: 
    ds_dir = pathlib.Path().home().joinpath("data", ds_dirname)
else: 
    ds_dir = coclico_data_dir.joinpath(ds_dirname, "data", "bgrm_delivery_18feb25")

if not ds_dir.exists():
    raise FileNotFoundError("Directory with data does not exist.")

# set ouput directory for front/end files
fe_output_dir = ds_dir.parent.parent.joinpath("front_end_data", "be_stats")


In [109]:
parquet_file = ds_dir.joinpath("numberbat200225.parquet")

be_stats = gpd.read_parquet(parquet_file)

# Change the direction of slashes
be_stats = be_stats.rename(columns=lambda x: x.replace('/', '\\'))

be_stats

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,LOW_DEFENDED_MAPS\static\SSP585\2100\telecom\rel_affected,LOW_DEFENDED_MAPS\static\SSP585\2100\telecom\rel_unaffected,LOW_DEFENDED_MAPS\static\SSP585\2100\gas\abs_affected,LOW_DEFENDED_MAPS\static\SSP585\2100\gas\abs_unaffected,LOW_DEFENDED_MAPS\static\SSP585\2100\gas\rel_affected,LOW_DEFENDED_MAPS\static\SSP585\2100\gas\rel_unaffected,LOW_DEFENDED_MAPS\static\SSP585\2100\total\abs_affected,LOW_DEFENDED_MAPS\static\SSP585\2100\total\abs_unaffected,LOW_DEFENDED_MAPS\static\SSP585\2100\total\rel_affected,LOW_DEFENDED_MAPS\static\SSP585\2100\total\rel_unaffected
0,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11960,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11961,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11962,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11963,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [92]:
11965 * 3 * 4 * 3 * 5

2153700

In [94]:
2010120/3/4/3/5

11167.333333333332

In [74]:
# Print all columns
for col in be_stats.columns:
    print(col)

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
UNDEFENDED_MAPS\static\SSP245\2010\buildings\abs_affected
UNDEFENDED_MAPS\static\SSP245\2010\buildings\abs_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\buildings\rel_affected
UNDEFENDED_MAPS\static\SSP245\2010\buildings\rel_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\healthcare\abs_affected
UNDEFENDED_MAPS\static\SSP245\2010\healthcare\abs_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\healthcare\rel_affected
UNDEFENDED_MAPS\static\SSP245\2010\healthcare\rel_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\oil\abs_affected
UNDEFENDED_MAPS\static\SSP245\2010\oil\abs_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\oil\rel_affected
UNDEFENDED_MAPS\static\SSP245\2010\oil\rel_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\power\abs_affected
UNDEFENDED_MAPS\static\SSP245\2010\power\abs_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\power\rel_affected
UNDEFENDED_MAPS\static\SSP245\2010\power\rel_unaffec

In [75]:
# List all the last elements of the columns containing 'MAPS'
map_cols = be_stats.columns[be_stats.columns.str.contains('MAPS')]

building_type = []

for col in map_cols:
    building_type.append(col.split('\\')[-2])

# Get unique building type
building_type = list(set(building_type))

building_type


['oil',
 'buildings',
 'power',
 'water',
 'air',
 'total',
 'road',
 'healthcare',
 'education',
 'gas',
 'wastewater',
 'telecom',
 'rail']

In [76]:
# # Store the data in a new parquet file
be_stats.to_parquet(fe_output_dir.joinpath("be_stats.parquet"))

In [110]:
# Drop all columns except for the first 11 and the columns containing 'total'
# Get the first 11 columns
columns_to_keep = be_stats.columns[:12].tolist()

# Keep columns that contain "total"
columns_with_total = [col for col in be_stats.columns if "total" in col]

# Final selection
be_stats = be_stats[columns_to_keep + columns_with_total]

# Get rid of the element "total" in the column names
be_stats.columns = be_stats.columns.str.replace('total\\', '')

# Print number of columns
len([be_stats.columns])

for col in be_stats.columns:
    print(col)

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
UNDEFENDED_MAPS\static\SSP245\2010\abs_affected
UNDEFENDED_MAPS\static\SSP245\2010\abs_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\rel_affected
UNDEFENDED_MAPS\static\SSP245\2010\rel_unaffected
UNDEFENDED_MAPS\static\SSP126\2010\abs_affected
UNDEFENDED_MAPS\static\SSP126\2010\abs_unaffected
UNDEFENDED_MAPS\static\SSP126\2010\rel_affected
UNDEFENDED_MAPS\static\SSP126\2010\rel_unaffected
UNDEFENDED_MAPS\static\SSP585\2010\abs_affected
UNDEFENDED_MAPS\static\SSP585\2010\abs_unaffected
UNDEFENDED_MAPS\static\SSP585\2010\rel_affected
UNDEFENDED_MAPS\static\SSP585\2010\rel_unaffected
HIGH_DEFENDED_MAPS\static\SSP245\2010\abs_affected
HIGH_DEFENDED_MAPS\static\SSP245\2010\abs_unaffected
HIGH_DEFENDED_MAPS\static\SSP245\2010\rel_affected
HIGH_DEFENDED_MAPS\static\SSP245\2010\rel_unaffected
HIGH_DEFENDED_MAPS\static\SSP126\2010\abs_affected
HIGH_DEFENDED_MAPS\static\SSP126\2010\abs_unaffected
HIGH

In [125]:
map_type = []
rp = []
scen = []
year = []

for col in be_stats.columns[12:]:
    map_type.append(col.split('\\')[0])
    rp.append(col.split('\\')[1])
    scen.append(col.split('\\')[2])
    year.append(col.split('\\')[3])

# Get unique values
map_type = list(set(map_type))
rp = list(set(rp))
scen = list(set(scen))
year = list(set(year))

map_type, rp, scen, year

(['HIGH_DEFENDED_MAPS', 'LOW_DEFENDED_MAPS', 'UNDEFENDED_MAPS'],
 ['100', 'static', '1000', '1'],
 ['High_End', 'SSP245', 'SSP585', 'SSP126'],
 ['2030', '2150', '2050', '2100', '2010'])

In [124]:
for col in be_stats.columns[12:]:
    print(col)

UNDEFENDED_MAPS\static\SSP245\2010\abs_affected
UNDEFENDED_MAPS\static\SSP245\2010\abs_unaffected
UNDEFENDED_MAPS\static\SSP245\2010\rel_affected
UNDEFENDED_MAPS\static\SSP245\2010\rel_unaffected
UNDEFENDED_MAPS\static\SSP126\2010\abs_affected
UNDEFENDED_MAPS\static\SSP126\2010\abs_unaffected
UNDEFENDED_MAPS\static\SSP126\2010\rel_affected
UNDEFENDED_MAPS\static\SSP126\2010\rel_unaffected
UNDEFENDED_MAPS\static\SSP585\2010\abs_affected
UNDEFENDED_MAPS\static\SSP585\2010\abs_unaffected
UNDEFENDED_MAPS\static\SSP585\2010\rel_affected
UNDEFENDED_MAPS\static\SSP585\2010\rel_unaffected
HIGH_DEFENDED_MAPS\static\SSP245\2010\abs_affected
HIGH_DEFENDED_MAPS\static\SSP245\2010\abs_unaffected
HIGH_DEFENDED_MAPS\static\SSP245\2010\rel_affected
HIGH_DEFENDED_MAPS\static\SSP245\2010\rel_unaffected
HIGH_DEFENDED_MAPS\static\SSP126\2010\abs_affected
HIGH_DEFENDED_MAPS\static\SSP126\2010\abs_unaffected
HIGH_DEFENDED_MAPS\static\SSP126\2010\rel_affected
HIGH_DEFENDED_MAPS\static\SSP126\2010\rel_unaffec

In [87]:
# For the front-end the there is a maximum amount of rows (255) that is allowed. For that reason we rework the dataset.
# The variables map_type, return_period, scenario and time will be added in a column. Strechting the dataset in the vertical direction.
df = be_stats.copy()

# Identify base columns that remain unchanged
base_columns = [
    "GISCO_ID", "CNTR_CODE", "nuts_2", "nuts_3", "LAU_ID", "LAU_NAME", 
    "POP_2020", "POP_DENS_2", "AREA_KM2", "YEAR", "FID", "geometry"
]

# Identify dynamic columns (all non-base columns)
variable_columns = [col for col in df.columns if col not in base_columns]

# Extract components from column names
df_extracted = pd.DataFrame([col.split('\\') for col in variable_columns], 
                            columns=["map_type", "return_period", "scenario", "time", "impact_type"])
df_extracted["original_column"] = variable_columns  # Keep track of original names

# Melt dataframe to long format
df_long = df.melt(id_vars=base_columns, value_vars=variable_columns, var_name="original_column", value_name="value")

# Merge extracted components directly into the long dataframe
df_long = df_long.merge(df_extracted, on="original_column").drop(columns=["original_column"])

# Pivot to create separate columns for impact types
df_wide = df_long.pivot(index=base_columns + ["map_type", "return_period", "scenario", "time"], 
                        columns="impact_type", 
                        values="value").reset_index()

# Remove column index name
df_wide.columns.name = None


be_stats_gpkg = df_wide.copy()

In [127]:
# Convert to geodataframe
be_stats_gpkg = gpd.GeoDataFrame(be_stats_gpkg, geometry="geometry")

# Rename FID to LAU_FID
be_stats_gpkg.rename(columns={'FID': 'LAU_FID'}, inplace=True)

# Drop all rows with time == 2150 or scenario == 'High_End'
be_stats_gpkg = be_stats_gpkg[(be_stats_gpkg.time != '2150') & (be_stats_gpkg.scenario != 'High_End')]

be_stats_gpkg

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,LAU_FID,geometry,map_type,return_period,scenario,time,abs_affected,abs_unaffected,rel_affected,rel_unaffected
2,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,AL_AL121,"POLYGON ((5125734.369 2084986.170, 5125441.232...",HIGH_DEFENDED_MAPS,1,SSP126,2010,17541.0,23342.0,0.429054,0.570946
3,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,AL_AL121,"POLYGON ((5125734.369 2084986.170, 5125441.232...",HIGH_DEFENDED_MAPS,1,SSP126,2030,17834.0,23049.0,0.436220,0.563780
4,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,AL_AL121,"POLYGON ((5125734.369 2084986.170, 5125441.232...",HIGH_DEFENDED_MAPS,1,SSP126,2050,18299.0,22584.0,0.447594,0.552406
5,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,AL_AL121,"POLYGON ((5125734.369 2084986.170, 5125441.232...",HIGH_DEFENDED_MAPS,1,SSP126,2100,19510.0,21373.0,0.477215,0.522785
6,AL_AL121,AL,AL01,AL012,AL121,Durrës,175110.0,517.362360,338.466834,2019,AL_AL121,"POLYGON ((5125734.369 2084986.170, 5125441.232...",HIGH_DEFENDED_MAPS,1,SSP245,2010,17541.0,23342.0,0.429054,0.570946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010115,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,UK_W06000023,"POLYGON ((3441623.655 3388948.562, 3441917.827...",UNDEFENDED_MAPS,static,SSP245,2100,3.0,731.0,0.004087,0.995913
2010116,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,UK_W06000023,"POLYGON ((3441623.655 3388948.562, 3441917.827...",UNDEFENDED_MAPS,static,SSP585,2010,3.0,731.0,0.004087,0.995913
2010117,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,UK_W06000023,"POLYGON ((3441623.655 3388948.562, 3441917.827...",UNDEFENDED_MAPS,static,SSP585,2030,3.0,731.0,0.004087,0.995913
2010118,UK_W06000023,UK,UKL2,UKL24,W06000023,Powys,132447.0,25.479361,5198.207327,2020,UK_W06000023,"POLYGON ((3441623.655 3388948.562, 3441917.827...",UNDEFENDED_MAPS,static,SSP585,2050,3.0,731.0,0.004087,0.995913


In [130]:
1722960/3/4/3/4

11965.0

In [131]:
# Write to geopackage
be_stats_gpkg.to_file(fe_output_dir.joinpath("be_stats.gpkg"), driver="GPKG")

In [90]:
# Only get unique map_columns
maps_columns = list(set([str(Path(col).parent) for col in be_stats.columns if "MAPS" in col]))

for map_column in maps_columns:
    
    print(map_column)
    # Select all column names containing the current map column
    cur_stats = be_stats.loc[:, be_stats.columns.str.startswith(map_column)]

    # split column names at backslashes, only keep last element
    cur_stats.columns = cur_stats.columns.str.split("\\").str[-1]

    # Add the cur stats to the first 11 columns of pp_stats
    pp_gpkg = pd.concat([be_stats.iloc[:, :12], cur_stats], axis=1)

    # Set FID to LAU_FID for storing as geopackage
    pp_gpkg.rename(columns={'FID': 'LAU_FID'}, inplace=True)

    # Set geopackage file name
    scen_name = map_column.replace("\\", "_")

    vars = map_column.split("\\")
    map_type = vars[0]
    rp = vars[1]
    scen = vars[2]
    year = vars[3]

    gpkg_fp = fe_output_dir.joinpath("map_stats", map_type,rp,scen)
    gpkg_fp.mkdir(parents=True, exist_ok=True)

    gpkg_file = gpkg_fp.joinpath(f"be_stats_{scen_name}.gpkg")

    # Write to geopackage
    pp_gpkg.to_file(str(gpkg_file), layer=f'be_stats_{scen_name}', driver='GPKG')

    # Write to parquet
    pp_gpkg.to_parquet(gpkg_fp.joinpath(f"be_stats_{scen_name}.parquet"))

LOW_DEFENDED_MAPS\1000\High_End\2150
UNDEFENDED_MAPS\1\SSP585\2050
HIGH_DEFENDED_MAPS\1000\SSP245\2050
HIGH_DEFENDED_MAPS\1000\SSP245\2100
HIGH_DEFENDED_MAPS\static\High_End\2100
HIGH_DEFENDED_MAPS\1\SSP585\2030
UNDEFENDED_MAPS\static\SSP245\2010
LOW_DEFENDED_MAPS\1000\SSP585\2010
LOW_DEFENDED_MAPS\1\SSP245\2050
UNDEFENDED_MAPS\1000\SSP126\2100
UNDEFENDED_MAPS\static\SSP245\2100
UNDEFENDED_MAPS\static\SSP245\2050
LOW_DEFENDED_MAPS\1\SSP585\2030
HIGH_DEFENDED_MAPS\static\SSP585\2100
HIGH_DEFENDED_MAPS\static\High_End\2150
HIGH_DEFENDED_MAPS\static\SSP245\2010
HIGH_DEFENDED_MAPS\100\SSP245\2100
UNDEFENDED_MAPS\static\SSP126\2010
LOW_DEFENDED_MAPS\static\SSP245\2010
UNDEFENDED_MAPS\1000\SSP126\2010
UNDEFENDED_MAPS\static\SSP126\2030
LOW_DEFENDED_MAPS\1000\SSP585\2030
HIGH_DEFENDED_MAPS\1\SSP245\2010
LOW_DEFENDED_MAPS\1\SSP245\2100
HIGH_DEFENDED_MAPS\static\SSP126\2050
LOW_DEFENDED_MAPS\static\SSP245\2100
UNDEFENDED_MAPS\static\SSP585\2010
HIGH_DEFENDED_MAPS\100\High_End\2150
LOW_DEFENDED_